## Сountry Сheck

[] - iOS, [] - Android
-----------------------
- [-] Austria
- [ ] Belgium
- [ ] Croatia
- [ ] Czech Republic
- [ ] Denmark
- [x],[x] Estonia
- [ ] Finland
- [ ] France
- [ ] Germany
- [ ] Greece

In [18]:
from serpapi import GoogleSearch
from dotenv import load_dotenv
import json
import os

load_dotenv()
api_key = os.getenv("API_KEY")

In [ ]:
search = GoogleSearch({
    "engine": "google_jobs",
    "q": "Android developer", 
    "location": "Estonia",
    "api_key": api_key
  })
result = search.get_dict()

file_path = r"Test\Belgium.json"  

# Saving JSON to a file
with open(file_path, "w", encoding="utf-8") as file:
    json.dump(result, file, ensure_ascii=False, indent=4)

Version 1:
search = GoogleSearch({
    "engine": "google_jobs",
    "q": "Job title", 
    "location": "Country",
    "api_key": api_key
  })